# Simulated fMRI Feature Extraction

This notebook demonstrates simulating and extracting features from fMRI (functional Magnetic Resonance Imaging) data for brain-music research.

## Background
- **fMRI** measures blood oxygen level-dependent (BOLD) signals
- **Spatial resolution**: ~2-3mm (excellent)
- **Temporal resolution**: ~1-2s (limited by hemodynamic response)
- **Coverage**: Whole brain

## Use Cases
- Localizing creativity-related brain regions
- Identifying music-responsive networks
- Studying functional connectivity patterns

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal, stats
from scipy.ndimage import gaussian_filter
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

print("Libraries loaded successfully")

## 1. Simulate fMRI Time Series

We'll simulate BOLD responses for different brain regions during a music listening task.

In [ ]:
# Parameters
TR = 2.0  # Repetition time in seconds (typical fMRI)
duration = 300  # 5 minutes scan
n_timepoints = int(duration / TR)
time = np.arange(n_timepoints) * TR

# Define regions of interest (ROIs)
roi_names = [
    'Auditory Cortex',
    'Default Mode Network',
    'Executive Control Network',
    'Reward System'
]

print(f"Simulating fMRI scan: {n_timepoints} timepoints, TR={TR}s")
print(f"ROIs: {', '.join(roi_names)}")

In [ ]:
def create_hrf(times, delay=6, undershoot_delay=16):
    """Create canonical hemodynamic response function"""
    hrf = np.zeros_like(times)
    hrf = (times ** (delay - 1) * np.exp(-times) / np.math.factorial(delay - 1) -
           0.35 * times ** (undershoot_delay - 1) * np.exp(-times) / 
           np.math.factorial(undershoot_delay - 1))
    hrf[times < 0] = 0
    return hrf / hrf.max()

# Create HRF for convolution
hrf_time = np.arange(0, 30, TR)
hrf = create_hrf(hrf_time)

# Plot HRF
plt.figure(figsize=(10, 4))
plt.plot(hrf_time, hrf, linewidth=2)
plt.xlabel('Time (s)')
plt.ylabel('Response')
plt.title('Canonical Hemodynamic Response Function (HRF)')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Simulate neural activity for different conditions
# Condition: music listening with creative engagement

# Auditory cortex: sustained activation during music
auditory_neural = np.ones(n_timepoints) + 0.3 * np.random.randn(n_timepoints)

# Default mode network: increases during creative/imaginative moments
creative_moments = np.random.choice([0, 1], size=n_timepoints, p=[0.7, 0.3])
dmn_neural = creative_moments + 0.2 * np.random.randn(n_timepoints)

# Executive control: periodic engagement for analytical listening
executive_neural = 0.5 * np.sin(2 * np.pi * time / 60) + 0.3 * np.random.randn(n_timepoints)
executive_neural[executive_neural < 0] = 0

# Reward system: spikes during particularly enjoyable moments
reward_spikes = np.zeros(n_timepoints)
spike_times = np.random.choice(n_timepoints, size=10, replace=False)
reward_spikes[spike_times] = np.random.uniform(1, 2, size=10)
reward_neural = reward_spikes + 0.2 * np.random.randn(n_timepoints)

# Convolve with HRF to get BOLD signals
bold_signals = np.zeros((len(roi_names), n_timepoints))
neural_signals = [auditory_neural, dmn_neural, executive_neural, reward_neural]

for i, neural in enumerate(neural_signals):
    bold = np.convolve(neural, hrf, mode='same')
    # Add scanner noise and drift
    drift = np.linspace(0, 0.5, n_timepoints)
    noise = 0.1 * np.random.randn(n_timepoints)
    bold_signals[i] = bold + drift + noise

print("✓ fMRI BOLD signals simulated")

In [ ]:
# Visualize simulated fMRI signals
fig, axes = plt.subplots(4, 1, figsize=(14, 10))

for i, (ax, name) in enumerate(zip(axes, roi_names)):
    ax.plot(time, bold_signals[i], linewidth=1.5)
    ax.set_ylabel('BOLD Signal')
    ax.set_title(f'{name}')
    ax.grid(alpha=0.3)
    
axes[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()

## 2. Feature Extraction

Extract meaningful features from fMRI data for brain-music mapping.

In [ ]:
def extract_fmri_features(bold_signals, roi_names, window_size=10):
    """Extract features from fMRI BOLD signals"""
    features = {}
    n_rois, n_timepoints = bold_signals.shape
    
    # 1. Mean activation per ROI
    features['mean_activation'] = np.mean(bold_signals, axis=1)
    
    # 2. Temporal variance (variability)
    features['temporal_variance'] = np.var(bold_signals, axis=1)
    
    # 3. Peak activation
    features['peak_activation'] = np.max(bold_signals, axis=1)
    
    # 4. Functional connectivity (correlation between ROIs)
    features['connectivity_matrix'] = np.corrcoef(bold_signals)
    
    # 5. Windowed features (for time-varying analysis)
    n_windows = n_timepoints // window_size
    windowed_mean = np.zeros((n_rois, n_windows))
    
    for i in range(n_windows):
        start = i * window_size
        end = (i + 1) * window_size
        windowed_mean[:, i] = np.mean(bold_signals[:, start:end], axis=1)
    
    features['windowed_activation'] = windowed_mean
    
    return features

# Extract features
features = extract_fmri_features(bold_signals, roi_names)

print("Extracted features:")
for key, value in features.items():
    if isinstance(value, np.ndarray):
        print(f"  {key}: shape {value.shape}")
    else:
        print(f"  {key}: {value}")

In [ ]:
# Visualize features
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Mean activation
axes[0, 0].bar(roi_names, features['mean_activation'])
axes[0, 0].set_ylabel('Mean BOLD Signal')
axes[0, 0].set_title('Mean Activation per ROI')
axes[0, 0].tick_params(axis='x', rotation=45)

# Temporal variance
axes[0, 1].bar(roi_names, features['temporal_variance'])
axes[0, 1].set_ylabel('Variance')
axes[0, 1].set_title('Temporal Variability')
axes[0, 1].tick_params(axis='x', rotation=45)

# Functional connectivity
im = axes[1, 0].imshow(features['connectivity_matrix'], cmap='coolwarm', vmin=-1, vmax=1)
axes[1, 0].set_xticks(range(len(roi_names)))
axes[1, 0].set_yticks(range(len(roi_names)))
axes[1, 0].set_xticklabels(roi_names, rotation=45, ha='right')
axes[1, 0].set_yticklabels(roi_names)
axes[1, 0].set_title('Functional Connectivity Matrix')
plt.colorbar(im, ax=axes[1, 0])

# Windowed activation (first ROI)
axes[1, 1].plot(features['windowed_activation'][0], marker='o')
axes[1, 1].set_xlabel('Time Window')
axes[1, 1].set_ylabel('Mean Activation')
axes[1, 1].set_title(f'Windowed Activation: {roi_names[0]}')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Dimensionality Reduction for Latent Mapping

Reduce high-dimensional fMRI features for mapping to music latent spaces.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Prepare feature matrix (combine all features)
# For this example, use windowed activation across all ROIs
feature_matrix = features['windowed_activation'].T  # Shape: (n_windows, n_rois)

# Standardize features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(feature_matrix)

# Apply PCA
pca = PCA(n_components=2)
features_pca = pca.fit_transform(features_scaled)

print(f"Original feature shape: {feature_matrix.shape}")
print(f"Reduced feature shape: {features_pca.shape}")
print(f"Explained variance ratio: {pca.explained_variance_ratio_}")
print(f"Total variance explained: {pca.explained_variance_ratio_.sum():.2%}")

In [ ]:
# Visualize PCA projection
plt.figure(figsize=(10, 6))
scatter = plt.scatter(features_pca[:, 0], features_pca[:, 1], 
                     c=np.arange(len(features_pca)), cmap='viridis', s=100)
plt.colorbar(scatter, label='Time Window')
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)')
plt.title('fMRI Features in PCA Space (Trajectory Over Time)')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## Summary

### What We Did
1. Simulated realistic fMRI BOLD signals for music-related brain regions
2. Extracted multiple feature types:
   - Mean activation levels
   - Temporal variability
   - Functional connectivity
   - Time-windowed activation patterns
3. Reduced dimensionality for latent space mapping

### Key Insights
- fMRI provides **spatial specificity** - which regions are active
- **Connectivity patterns** reveal network-level dynamics
- **Temporal dynamics** are limited (slow hemodynamic response)
- PCA reveals **low-dimensional structure** in brain activity

### Next Steps
1. Map these features to music model latent spaces (see `06_latent_space_mapping.ipynb`)
2. Compare with EEG features for complementary information
3. Explore more sophisticated feature extraction (GLM, connectivity metrics)

### Limitations
- Simulated data lacks real brain complexity
- Real fMRI requires artifact removal, motion correction, preprocessing
- Individual differences not captured here
- Real-time fMRI challenging due to processing requirements